In [ ]:
import os
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoProcessor,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

In [ ]:

# import wandb

# Initialize wandb for tracking
# wandb.init(project="qwen-patchcamelyon", name="qwen2-lora-patchcamelyon")

# Load the dataset
dataset = load_dataset("yashasvikan/blip2-annotated-patchcamelyon")

# Split dataset into train and test
dataset = dataset.shuffle(seed=42)
split_dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Load model, tokenizer, and processor
model_id = "Qwen/Qwen2-7B-VL"
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id)

# Configure quantization for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Configure LoRA
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

# Create a custom data collator for vision-language inputs
class VisionTextDataCollator:
    def __init__(self, processor, tokenizer, max_length=512):
        self.processor = processor
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, examples):
        images = [example["image"] for example in examples]
        captions = [example["caption"] for example in examples]
        labels = [example["label"] for example in examples]

        # Process images
        vision_inputs = self.processor(images=images, return_tensors="pt")

        # Format prompts with labels
        prompts = [
            f"<image>\nDescribe this histopathology image and determine if it shows cancer metastasis. Label: {1 if label == 1 else 0}\n"
            for label in labels
        ]

        # Tokenize text
        text_inputs = self.tokenizer(
            prompts,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Combine inputs
        batch = {
            "input_ids": text_inputs.input_ids,
            "attention_mask": text_inputs.attention_mask,
            "pixel_values": vision_inputs.pixel_values,
            "labels": torch.tensor(labels)
        }

        return batch

# Initialize data collator
data_collator = VisionTextDataCollator(processor, tokenizer)

# Training arguments
training_args = TrainingArguments(
    output_dir="./qwen-patchcamelyon-results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    num_train_epochs=3,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    remove_unused_columns=False,
)

# Initialize the SFT Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation results: {results}")

# Perform inference on test set
def predict(batch):
    with torch.no_grad():
        inputs = data_collator([batch])
        for k, v in inputs.items():
            if isinstance(v, torch.Tensor):
                inputs[k] = v.to(model.device)

        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        return {"predictions": predictions}

predictions = []
true_labels = []

for i in range(len(test_dataset)):
    result = predict(test_dataset[i])
    predictions.append(result["predictions"][0])
    true_labels.append(test_dataset[i]["label"])

# Calculate metrics
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')
conf_matrix = confusion_matrix(true_labels, predictions)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Confusion Matrix:\n{conf_matrix}")

# # Log final metrics to wandb
# wandb.log({
#     "test_accuracy": accuracy,
#     "test_precision": precision,
#     "test_recall": recall,
#     "test_f1": f1,
# })

# Save the fine-tuned model
trainer.model.save_pretrained("./qwen-patchcamelyon-final")
tokenizer.save_pretrained("./qwen-patchcamelyon-final")


In [ ]:
!pip3 install bitsandbytes peft trl

In [ ]:
def get_dataset():
  from datasets import load_dataset, DatasetDict

  # Load the full dataset (this may take time and space)
  ds = load_dataset("zacharielegault/PatchCamelyon")

  # Access the 'train' split – PatchCamelyon only has one 'train' split, no 'test'
  full_dataset = ds['train']
  total_len = len(full_dataset)  # Should be around 327,000+

  # Define index ranges
  train_indices = list(range(0, 750)) + list(range(131072, 131820))
  val_indices = list(range(750, 1000)) + list(range(131820, 132072))
  test_size = int(0.01 * total_len)
  test_indices = list(range(total_len - test_size, total_len))  # last 1%

  # Apply slicing using `select`
  train_dataset = full_dataset.select(train_indices)
  val_dataset = full_dataset.select(val_indices)
  test_dataset = full_dataset.select(test_indices)

  # Optional: Bundle as a DatasetDict
  custom_splits = DatasetDict({
      "train": train_dataset,
      "validation": val_dataset,
      "test": test_dataset
  })
  return custom_splits

In [ ]:
def get_test_dataset():
  from datasets import load_dataset, DatasetDict

  # Load the full dataset (this may take time and space)
  ds = load_dataset("zacharielegault/PatchCamelyon")

  # Access the 'train' split – PatchCamelyon only has one 'train' split, no 'test'
  full_dataset = ds['test']
  total_len = len(full_dataset)  # Should be around 26,000+

  # Define index ranges
  test_indices = list(range(0, 150)) + list(range(20000, 20150))

  # Apply slicing using `select`
  test_dataset = full_dataset.select(test_indices)

  # Optional: Bundle as a DatasetDict
  custom_splits = DatasetDict({
      "test": test_dataset
  })
  return custom_splits

In [ ]:
test_dataset = get_test_dataset()["test"]

In [ ]:
ds = get_dataset()
ds["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96>,
 'label': 0}

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Load the BLIP model and processor (do this only once)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
model.eval()

# If GPU is available, use it
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_blipcaption(example):
    """
    Given a dictionary with 'image': PIL.Image and 'label': int,
    return the caption string generated by BLIP.
    """
    image = example['image']
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        output_ids = model.generate(**inputs)

    caption = processor.decode(output_ids[0], skip_special_tokens=True)
    return {
        "image": example["image"],
        "label": example["label"],
        "caption": caption
    }



Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
test_dataset = ds['test']
test_dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96>,
 'label': 1}

In [ ]:
test_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 2621
})

In [ ]:
def caption_test_data(test_dataset):
  test_dataset = test_dataset.map(get_blipcaption, batched=False)
  return test_dataset
test_dataset = caption_test_data(test_dataset)

Map:   0%|          | 0/2621 [00:00<?, ? examples/s]

In [ ]:
test_dataset

Dataset({
    features: ['image', 'label', 'caption'],
    num_rows: 2621
})

In [ ]:
# === Run captioning on all splits ===
def caption_all_splits(dataset_dict):
    new_dataset = DatasetDict()
    for split in dataset_dict:
        print(f"Generating captions for split: {split}")
        new_dataset[split] = dataset_dict[split].map(get_blipcaption, batched=False)
    return new_dataset

# === Run the thing ===
captioned_dataset = caption_all_splits(ds)

NameError: name 'DatasetDict' is not defined

In [ ]:
captioned_dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'caption'],
        num_rows: 1498
    })
    validation: Dataset({
        features: ['image', 'label', 'caption'],
        num_rows: 502
    })
    test: Dataset({
        features: ['image', 'label', 'caption'],
        num_rows: 2621
    })
})

In [ ]:
import os
# os.environ["WANDB_DISABLED"] = "true"

from datasets import load_dataset
import torch
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer

import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"
EPOCHS = 1
BATCH_SIZE = 1
GRADIENT_CHECKPOINTING = True,  # Tradeoff between memory efficiency and computation time.
USE_REENTRANT = False,
OPTIM = "paged_adamw_32bit"
LEARNING_RATE = 2e-5
LOGGING_STEPS = 50
EVAL_STEPS = 50
SAVE_STEPS = 50
EVAL_STRATEGY = "steps"
SAVE_STRATEGY = "steps"
METRIC_FOR_BEST_MODEL="eval_loss"
LOAD_BEST_MODEL_AT_END=True
MAX_GRAD_NORM = 1
WARMUP_STEPS = 0
DATASET_KWARGS={"skip_prepare_dataset": True} # We have to put for VLMs
REMOVE_UNUSED_COLUMNS = False # VLM thing
MAX_SEQ_LEN=1
NUM_STEPS = (283 // BATCH_SIZE) * EPOCHS
print(f"NUM_STEPS: {NUM_STEPS}")

Using device: cuda
NUM_STEPS: 283


In [ ]:
system_message = """You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images.
Your task is to process and extract if it is cancerous image or not,
leveraging multimodal understanding to provide accurate and contextually relevant information.
Return 1 if there is cancer tumour, 0 if not.
Return 1 or 0."""

def format_data(sample):
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": sample["image"],
                },
                {
                    "type": "text",
                    "text": sample["caption"] + "\nUsing the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise.",
                },
            ],
        }
    ]

In [ ]:

train_dataset, eval_dataset, test_dataset = captioned_dataset['train'], captioned_dataset['validation'], captioned_dataset['test']

In [ ]:
print(len(train_dataset))
print("-"*30)
print(train_dataset)
print("-"*30)
print(train_dataset[0])
print("-"*30)
train_dataset, eval_dataset, test_dataset = captioned_dataset['train'], captioned_dataset['validation'], captioned_dataset['test']
train_dataset = [format_data(sample) for sample in train_dataset]
eval_dataset = [format_data(sample) for sample in eval_dataset]
test_dataset = [format_data(sample) for sample in test_dataset]

print(len(train_dataset))
print("-"*30)
print(train_dataset[0])
print("-"*30)
print(len(test_dataset))
print("-"*30)
print(test_dataset[0])

1498
------------------------------
Dataset({
    features: ['image', 'label', 'caption'],
    num_rows: 1498
})
------------------------------
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96 at 0x79932A3B3D90>, 'label': 0, 'caption': 'purple glitter con con con con con con con con con con con con con con con con con con'}
------------------------------
1498
------------------------------
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. \nYour task is to process and extract if it is cancerous image or not, \nleveraging multimodal understanding to provide accurate and contextually relevant information.\nReturn 1 if there is cancer tumour, 0 if not.\nReturn 1 or 0.'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96 at 0x79932A3B2B10>}, {'type': 'text', 'text': "purple glitter con con 

In [ ]:
sample_data = test_dataset[0]
sample_question = test_dataset[0][1]["content"][1]["text"]
sample_answer = test_dataset[0][2]["content"][0]["text"]
sample_image = test_dataset[0][1]["content"][0]["image"]

print(sample_question)
print(sample_answer)
sample_image

KeyError: 1

In [ ]:
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        use_cache=False
        )

else:
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        use_cache=False
        )

processor = Qwen2VLProcessor.from_pretrained(MODEL_ID)
processor.tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
def text_generator(sample_data):
    text = processor.apply_chat_template(
        sample_data[0:2], tokenize=False, add_generation_prompt=True
    )

    print(f"Prompt: {text}")
    print("-"*30)

    image_inputs = sample_data[1]["content"][0]["image"]

    inputs = processor(
        text=[text],
        images = image_inputs,
        return_tensors="pt",
        padding=False,
        truncation=False
    )
    inputs = inputs.to(device)

    generated_ids = model.generate(**inputs, max_new_tokens=MAX_SEQ_LEN)

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True
    )
    del inputs
    actual_answer = sample_data[2]["content"][0]["text"]
    import re
    match = re.search(r"\d(?!.*\d)", output_text[0])
    last_digit = match.group(0) if match else "?"

    return last_digit, actual_answer


generated_text, actual_answer = text_generator(sample_data)
print(f"Generated Answer: {generated_text}")
print(f"Actual Answer: {actual_answer}")

Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or not, 
leveraging multimodal understanding to provide accurate and contextually relevant information.
Return 1 if there is cancer tumour, 0 if not.
Return 1 or 0.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>a close up of a pink granite counter
Using the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise.<|im_end|>
<|im_start|>assistant

------------------------------
Generated Answer: 0
Actual Answer: 1


In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    target_modules=["q_proj", "q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

print(f"Before adapter parameters: {model.num_parameters()}")
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters() # After LoRA trainable parameters increases. Since we add adapter.

Before adapter parameters: 8291375616
trainable params: 2,523,136 || all params: 8,293,898,752 || trainable%: 0.0304


In [ ]:
training_args = SFTConfig(
    output_dir="./output",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_checkpointing=GRADIENT_CHECKPOINTING,
    learning_rate=LEARNING_RATE,
    logging_steps=LOGGING_STEPS,
    eval_steps=EVAL_STEPS,
    eval_strategy=EVAL_STRATEGY,
    save_strategy=SAVE_STRATEGY,
    save_steps=SAVE_STEPS,
    metric_for_best_model=METRIC_FOR_BEST_MODEL,
    load_best_model_at_end=LOAD_BEST_MODEL_AT_END,
    max_grad_norm=MAX_GRAD_NORM,
    warmup_steps=WARMUP_STEPS,
    dataset_kwargs=DATASET_KWARGS,
    max_seq_length=MAX_SEQ_LEN,
    remove_unused_columns = REMOVE_UNUSED_COLUMNS,
    optim=OPTIM,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
def tokenize_and_mask(example):
    prompt = processor.apply_chat_template(example["content"], tokenize=False)
    image = example["image"]

    # Tokenize text and image
    encoded = processor(text=prompt, images=image, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

    # Create labels: same as input_ids, except pad tokens = -100
    input_ids = encoded["input_ids"][0]
    labels = input_ids.clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100

    # Force label token to be the last one ("0" or "1")
    target_token = processor.tokenizer(str(example["label"]), add_special_tokens=False)["input_ids"]
    if len(target_token) != 1:
        raise ValueError(f"Label {example['label']} is not a single token")

    labels[-1] = target_token[0]  # overwrite last position with target
    return {
        "input_ids": input_ids,
        "attention_mask": encoded["attention_mask"][0],
        "pixel_values": encoded["pixel_values"][0],
        "labels": labels
    }


In [ ]:
collate_sample = [train_dataset[0], train_dataset[1]] # for batch size 2.

def collate_fn(batch):
  return {
       "input_ids": torch.stack([ex["input_ids"] for ex in batch]),
        "attention_mask": torch.stack([ex["attention_mask"] for ex in batch]),
        "pixel_values": torch.stack([ex["pixel_values"] for ex in batch]),
        "labels": torch.stack([ex["labels"] for ex in batch])}

collated_data = collate_fn(collate_sample)
print(collated_data.keys())  # dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'labels'])

dict_keys(['input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw', 'labels'])


In [ ]:
train_dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96>,
 'label': 0,
 'caption': 'purple glitter con con con con con con con con con con con con con con con con con con'}

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    processing_class=processor.tokenizer,

)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
print("-"*30)
print("Initial Evaluation")
metric = trainer.evaluate()
print(metric)
print("-"*30)

print("Training")
trainer.train()
print("-"*30)

------------------------------
Initial Evaluation


{'eval_loss': 4.272802352905273, 'eval_model_preparation_time': 0.0041, 'eval_runtime': 90.7918, 'eval_samples_per_second': 5.529, 'eval_steps_per_second': 5.529}
------------------------------
Training


Step,Training Loss,Validation Loss,Model Preparation Time
50,4.151200,3.902993,0.004100
100,3.616800,3.272860,0.004100
150,2.870700,2.395175,0.004100
200,1.916600,1.474343,0.004100
250,1.248100,1.072410,0.004100
300,0.978200,0.935614,0.004100
350,0.916600,0.901972,0.004100
400,0.869800,0.880267,0.004100
450,0.890000,0.861952,0.004100
500,0.866400,0.850142,0.004100


------------------------------


In [ ]:
trainer.save_model(training_args.output_dir)

In [ ]:
test_dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96>,
 'label': 0,
 'caption': 'imbauella cyosus, a type of the immune'}

In [ ]:
from tqdm import tqdm
import torch
import re

def extract_final_digit(text):
    match = re.search(r"[01](?!.*[01])", text.strip())
    return int(match.group(0)) if match else -1  # -1 = fallback for garbage outputs

def evaluate_model_on_test(test_dataset, processor, model):
    model.eval()
    model.to("cuda" if torch.cuda.is_available() else "cpu")

    correct = 0
    total = 0

    for example in tqdm(test_dataset):
        # 1. Generate text prompt
        text = processor.apply_chat_template(
            example[0:2], tokenize=False, add_generation_prompt=True
        )

        print(f"Prompt: {text}")
        print("-"*30)

        image_inputs = example[1]["content"][0]["image"]

        inputs = processor(
            text=[text],
            images = image_inputs,
            return_tensors="pt"
        )
        inputs = inputs.to(device)

        generated_ids = model.generate(**inputs, max_new_tokens=MAX_SEQ_LEN)

        output_text = processor.batch_decode(
            generated_ids, skip_special_tokens=True
        )
        del inputs
        actual_answer = example[2]["label"]
        import re
        match = re.search(r"\d(?!.*\d)", output_text[0])
        last_digit = match.group(0) if match else "?"

        gold = example[2]["label"]
        print(f"Response: {last_digit}")
        print(f"Gold: {gold}")
        print("-"*30)
        if int(last_digit) == int(gold):
            correct += 1
        total += 1

    accuracy = correct / total if total else 0
    return accuracy


In [ ]:
system_message = """You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images.
Your task is to process and extract if it is cancerous image or not,
leveraging multimodal understanding to provide accurate and contextually relevant information.
Return 1 if there is cancer tumour, 0 if not.
Return 1 or 0."""

def format_data(sample):
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": sample["image"],
                },
                {
                    "type": "text",
                    "text": sample["caption"] + "\nUsing the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise.",
                },
            ],
        },
        {
            "label": sample["label"]
        }
    ]

In [ ]:
# train_dataset = [format_data(sample) for sample in train_dataset]
# eval_dataset = [format_data(sample) for sample in eval_dataset]
test_dataset = [format_data(sample) for sample in test_dataset]


In [ ]:
test_dataset

[[{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. \nYour task is to process and extract if it is cancerous image or not, \nleveraging multimodal understanding to provide accurate and contextually relevant information.\nReturn 1 if there is cancer tumour, 0 if not.\nReturn 1 or 0.'}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96>},
    {'type': 'text',
     'text': "imbauella cyosus, a type of the immune\nUsing the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise."}]},
  {'label': 0}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. \nYour task is to process and extract if it is cancerous image or not, \nleveraging 

In [ ]:
import gc
import time
gc.collect()
torch.cuda.empty_cache()

In [ ]:
test_dataset

[[{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. \nYour task is to process and extract if it is cancerous image or not, \nleveraging multimodal understanding to provide accurate and contextually relevant information.\nReturn 1 if there is cancer tumour, 0 if not.\nReturn 1 or 0.'}]},
  {'role': 'user',
   'content': [{'type': 'image',
     'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=96x96>},
    {'type': 'text',
     'text': "imbauella cyosus, a type of the immune\nUsing the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise."}]},
  {'label': 0}],
 [{'role': 'system',
   'content': [{'type': 'text',
     'text': 'You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. \nYour task is to process and extract if it is cancerous image or not, \nleveraging 

In [ ]:
from transformers import AutoTokenizer, AutoProcessor, AutoModelForCausalLM

# Load model if not already
model = Qwen2VLForConditionalGeneration.from_pretrained(training_args.output_dir)
processor = Qwen2VLProcessor.from_pretrained(MODEL_ID)
processor.tokenizer.padding_side = "right"



Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:

accuracy = evaluate_model_on_test(test_dataset, processor, model)
print(f"Test Accuracy: {accuracy:.2%}")

  0%|          | 0/300 [00:00<?, ?it/s]

Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or not, 
leveraging multimodal understanding to provide accurate and contextually relevant information.
Return 1 if there is cancer tumour, 0 if not.
Return 1 or 0.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>imbauella cyosus, a type of the immune
Using the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise.<|im_end|>
<|im_start|>assistant

------------------------------


  1%|          | 2/300 [00:00<01:00,  4.90it/s]

Response: 0
Gold: 0
------------------------------
Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or not, 
leveraging multimodal understanding to provide accurate and contextually relevant information.
Return 1 if there is cancer tumour, 0 if not.
Return 1 or 0.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>a pink glitter confection with a white background
Using the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise.<|im_end|>
<|im_start|>assistant

------------------------------
Response: 0
Gold: 0
------------------------------
Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or not, 
leveraging multimodal understanding to prov

  1%|▏         | 4/300 [00:00<00:57,  5.15it/s]

...

Response: 0
Gold: 1
------------------------------
Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or not, 
leveraging multimodal understanding to provide accurate and contextually relevant information.
Return 1 if there is cancer tumour, 0 if not.
Return 1 or 0.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>a purple background with a white dot
Using the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise.<|im_end|>
<|im_start|>assistant

------------------------------
Response: 0
Gold: 1
------------------------------
Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or not, 
leveraging multimodal understanding to provide accurate 

100%|█████████▉| 299/300 [01:10<00:00,  5.17it/s]

Response: 0
Gold: 1
------------------------------
Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or not, 
leveraging multimodal understanding to provide accurate and contextually relevant information.
Return 1 if there is cancer tumour, 0 if not.
Return 1 or 0.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>a purple and white spec spec spec spec spec spec spec spec spec spec spec spec spec spec spec spec
Using the image and it's captoin, predict if there is cancer or not. Return 1 if there is cancer, 0 otherwise.<|im_end|>
<|im_start|>assistant

------------------------------
Response: 0
Gold: 1
------------------------------
Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, histopathology images. 
Your task is to process and extract if it is cancerous image or 

100%|██████████| 300/300 [01:11<00:00,  4.20it/s]

Response: 0
Gold: 1
------------------------------
Test Accuracy: 50.00%


In [ ]:
import gc
import time

# https://huggingface.co/learn/cookbook/en/fine_tuning_vlm_trl
def clear_memory():
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


clear_memory()

GPU allocated memory: 0.02 GB
GPU reserved memory: 0.07 GB


In [ ]:
if device == "cuda":
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        use_cache=True
        )

else:
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        MODEL_ID,
        use_cache=True
        )

processor = Qwen2VLProcessor.from_pretrained(MODEL_ID)
processor.tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
print(f"Before adapter parameters: {model.num_parameters()}")
model.load_adapter("./output")
print(f"After adapter parameters: {model.num_parameters()}")

Before adapter parameters: 8291375616
After adapter parameters: 8293898752


In [ ]:
generated_text, actual_answer = text_generator(sample_data)
print(f"Generated Answer: {generated_text}")
print(f"Actual Answer: {actual_answer}")

Prompt: <|im_start|>system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, describing, and interpreting visual data. 
Your task is to process and extract meaningful insights from images, videos, and visual patterns, 
leveraging multimodal understanding to provide accurate and contextually relevant information.<|im_end|>
<|im_start|>user
<|vision_start|><|image_pad|><|vision_end|>How many food item is shown in the bar graph?<|im_end|>
<|im_start|>assistant

------------------------------
Generated Answer: system
You are a highly advanced Vision Language Model (VLM), specialized in analyzing, describing, and interpreting visual data. 
Your task is to process and extract meaningful insights from images, videos, and visual patterns, 
leveraging multimodal understanding to provide accurate and contextually relevant information.
user
How many food item is shown in the bar graph?
assistant
There are 11 food items shown in the bar graph.
Actual Answer: 14
